# Riot API 

This analysis isn't endorsed by Riot Games and doesn't reflect the views or opinions of Riot Games or anyone officially involved in producing or managing Riot Games properties. Riot Games, and all associated properties are trademarks or registered trademarks of Riot Games, Inc.

REMINDER: API changes every couple of days, each hash changes along with it!

## To Do

1. (Current) Code experimentation
2. Running Collection for Dataset
3. Preparing Analysis

Priority
* Scheduling running of the collection code

* SQL alchemy 
    * Duplicate handling
    * [TODO] Logging just to see what's happening
    * [TODO] Sessions/Transactions?


* Environment Preparation (Isolating just the necessary libraries to run this code
    * Is there a Better way of showing the needed libaries?

* Workflow for analysis
    * Generic
        * Match ID
            * Username
                * Ranking
    * Targeted
        * Username
           * Match information of previous matches

## Goal

Showcase DataScience with limited Data

## Plan

1. API ingestion into clear columns
1. Database Setup
    1. Users (puuid, summonerid, username)
    1. Ranking Table (summoner revision date, rank, title, lp)
1. Database insertion of data
    1. How to cache this data for the future
1. Explaining more about Team Fight Tactics


## Ideas:
* 

## Links
* https://developer.riotgames.com/apis

* https://requests.readthedocs.io/en/latest/user/quickstart/


## Lessons Learned
* Python Requests .json() returns a dict that can be encoded to JSON (AKA deserialization) requires serialization to turn into proper JSON. Difference is that the deserialization creates a dictionary that uses single quotes. However JSON requires double quotes for strings! Can use json library function dumps to serialize, or just replace single quotations with double quotations. 

* SQL Alchemy 
    * Object Oriented Programming
    * Sessions and transactions for logging purposes
    * Mostly flexible backend. There are some backend specific functions that is not universal

* Github
    * Hiding API keys through gitignore

* Rate Limiting
    * Decorator Factories (Merging Decorators)
    
* Decision on Class Creation
    * Not everything deserves to be a class
    * In my own words: "Classes are best used as interfaces where interactivity of an object is the main goal"
    * For this project the data is just being pipelined to another part, theres no altering of a single object, they're being turned into a different object
    
    
    

# Classes 
classes for more structured code? Probably best for showcasing and the API Limter decorator
For best usage Requires more than 1 method....


objects & methods
* user
    * username
    * PUUID
    


* match

* user ranking information




* Requests from the API
    * Request "account info" given username
    * Request ranking information given summoner id (https://developer.riotgames.com/apis#tft-summoner-v1)
    * Request ranking information given rank and placement (https://developer.riotgames.com/apis#tft-league-v1)
        * Top 100 Challenger
        * Top 200 GM
        * Top 100 Master
        * Per Rank and Placement (Diamond-Bronze)
    * Request Match IDs given puuid and n latest matches
    * Request Match Information given Match IDs
    
* Inserts to the Database
    * Basically One For each table 

Questions:
* How do classes interact with each other, especially with piping methods into and out of different classes? 
* Are there other reused classes like user and match? (Maybe within the breakdown of the data from API to Database



Classes Relative to use-case
Use Case Scenario 1: Creating a database for analysis and ML Modeling
Steps
1. Query for a specific subset of users based on REGION and RANK
1. Store Match information (Duplicate Match IDs should be checked and skipped for optimization)
1. Store Ranking information (difficult, rank over time is not stored. Best way would be to query that data seperately every 30 mins) [Push to Next Version]


Use Case Scenario 2: Looking up users within my match
1. Query for match history based on manually inputted usernames
1. Store Match Database for Analysis
1. Run analysis ("this person's top 3 comps are ... and have ... TOP 4%. )"
1. Utilize "ML model" for prediction


# Overview

## Objective

## ETL Process
1. EXTRACT - API Configuration
1. TRANSFORM - Data Munging Userprofiles and TFT Matches 
1. LOAD - SQLAlchemy Insertion into Storage Database

## Data Analysis and Modeling
1. Comparisons per Rank and Region
    * Hypothesis
        * As Rank increases, competition becomes tighter
        * Different regions have noticably different playstyles/metas
    
    * Hypothetical Root Causes:
        * Rank - Higher rank will show less gold remaining when Health reaches 0
        * Rank - Higher rank will show shorter stages of the game
    
    * Hypothetical Solutions:
1. "What makes BOT4?" 
    * Hypothesis: Inaccurate player understanding of their comp powerlevel relative to other players
        * Playing too greedy when actually terminally weak
    * Hypothetical Root Causes:
        * 
    * Hypothetical Solutions:
        * Access damage dealt is above "some threshold" by "some stage in the game"

1. "What is the optimal setup for comps?"
    * Example: For Sharpshooters how much higher is the average placement for Teemo Chosen vs Vayne Chosen
1. Conditional Matches
    * If a match was close what are "good enough" comps
    * If contested, which comps do better
 
## Implementation for Competitive Usage
1. Basic analytics per User output, expose one-trick players
1. Suggest counter comps


## Summary

---

# ETL 

In [74]:
# https://pypi.org/project/ratelimit/
from ratelimit import limits, sleep_and_retry
#        > 20 requests every 1 seconds(s)        
#        > 100 requests every 2 minutes(s)
# 10 calls per minute
CALLS = 10
RATE_LIMIT = 60

# 1 calls per 2 seconds
CALLS_2 = 1 
RATE_LIMIT_2 = 2


# Make all the api calls limited based on rate
@sleep_and_retry
@limits(calls=CALLS, period=RATE_LIMIT)
def check_limit():
    ''' Empty function just to check for calls to API '''
    return

@sleep_and_retry
@limits(calls=CALLS_2, period=RATE_LIMIT_2)
def check_limit_2():
    ''' Empty function just to check for calls to API '''
    return

In [75]:
# Decorator factory to merge the decorators together!
def merged_decorator_with_args(CALLS, RATE_LIMIT, CALLS_2, RATE_LIMIT_2):
    # pass the arguments to the decorator factories and
    # obtain the actual decorators
    deco2 = limits(calls=CALLS, period=RATE_LIMIT)
    deco1 = limits(calls=CALLS_2, period=RATE_LIMIT_2)

    # create a function decorator that applies the two
    # decorators we just created
    def real_decorator(func):
        return deco2(deco1(func))

    return real_decorator

# Good start, will need to add-in sleep_and_retry
    # also will need to say what is not working data validation
# Will need to make the API calls functions... Maybe this should be a class
    # Technically these are different methods
        # Header with the API
        # Different links which are used for the library Requests request.get .... the URL and the argument
            # The JSON responses are broken down and then included in the the SQL insert
# One class for breaking the data down
# One class for inserting the database



In [82]:
def test():
    check_limit()
    check_limit_2()
    return "test"

def test2():
    check_limit()
    check_limit_2()
    return "test2"

In [102]:
@merged_decorator_with_args(CALLS, RATE_LIMIT, CALLS_2, RATE_LIMIT_2)
def test3():
    return "test3"

In [11]:
# Libraries
import requests

import pprint
pp = pprint.PrettyPrinter(depth=8)

# import json # Requests already converts JSON into dict

In [2]:
# Api Key Needs to be changed every couple of days!!!!
    #api_key = "EXAMPLE-NUMBERS-LETTERS-UPPERCASE-LOWERCASE"
# Header for API queries
%run config.py

In [3]:
# API response dictionary (human readable errors)
API_Responses = {
    200: "OK",
    400: "Bad request",
    401: "Unauthorized",
    403: "Forbidden",
    404: "Data not found",
    405: "Method not allowed",
    415: "Unsupported media type",
    429: "Rate limit exceeded",
    500: "Internal server error",
    502: "Bad gateway",
    503: "Service unavailable",
    504: "Gateway timeout"
}
# TODO Create error handling depending on API response


In [9]:
RANK = "DIAMOND" # PLATINUM, GOLD, SLIVER, BRONZE, IRON
TIER = "I" # I, II, III, IV
page = 1 #I guess 1 is the first page (ie highest level for that rank)

search_rank = "https://na1.api.riotgames.com/tft/league/v1/entries/{}/{}?page={}".format(RANK, TIER, page)
response_search_rank = requests.get(search_rank, headers=headers)
print(API_Responses[response_search_rank.status_code])

OK


In [12]:
pp.pprint(response_search_rank.json())


[{'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': '3ef1e7ef-b456-4755-8740-33acd56607c6',
  'leaguePoints': 0,
  'losses': 258,
  'queueType': 'RANKED_TFT',
  'rank': 'I',
  'summonerId': 'xzbLh06cSRLW7FKh6_-JPQB6U1ahWLYgy119kajYkXvXWPte',
  'summonerName': 'goopmeister',
  'tier': 'DIAMOND',
  'veteran': False,
  'wins': 47},
 {'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': 'b1a532b2-1b1f-4e89-858d-5523d26aff8c',
  'leaguePoints': 30,
  'losses': 84,
  'queueType': 'RANKED_TFT',
  'rank': 'I',
  'summonerId': 'uZ7cA_5-54lY8DiygqUiZV6NoctEgtj33XKD9DTneWcNSoQ1',
  'summonerName': 'gunning4you',
  'tier': 'DIAMOND',
  'veteran': False,
  'wins': 14},
 {'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': '3ec83a80-cf10-4085-90ed-0b39ad541604',
  'leaguePoints': 0,
  'losses': 58,
  'queueType': 'RANKED_TFT',
  'rank': 'I',
  'summonerId': 'rxtbC-88m9Ao1LxqZWGAxmsl9-7jOfxddZP65BQ18iUAsxQH',
  'summonerNa

  'veteran': False,
  'wins': 43},
 {'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': '6b3f1476-714b-441e-9dc0-40d5fa29316c',
  'leaguePoints': 0,
  'losses': 52,
  'queueType': 'RANKED_TFT',
  'rank': 'I',
  'summonerId': 'pFmhieoGMVLzmcY_bsSPAoJjZPD8ImcOcj481mxkvvEMvZKq',
  'summonerName': 'FROMTheSpacexx',
  'tier': 'DIAMOND',
  'veteran': False,
  'wins': 16},
 {'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': '3238e5e2-c984-479f-904f-dfd7ae185f25',
  'leaguePoints': 51,
  'losses': 264,
  'queueType': 'RANKED_TFT',
  'rank': 'I',
  'summonerId': 'cmpPPWJ2TiGjKb7sQdtu0fXV-H0U28V6N_CKapOb9f95g87S',
  'summonerName': 'Ansou',
  'tier': 'DIAMOND',
  'veteran': False,
  'wins': 56},
 {'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': 'b10f70f9-6303-4c35-b975-b7b31590203f',
  'leaguePoints': 33,
  'losses': 39,
  'queueType': 'RANKED_TFT',
  'rank': 'I',
  'summonerId': '13npjaPDkE8gQ2I3_lgrjzfMXBmGjiq

In [ ]:
# Breakdown the ranking per person

# which is faster by summonerName or summonerID

# [1 query] Get TIER and Rank of page 1 
# JSON -> List of names () [n = total length of name list]
# [n queries] List of names -> last 20 matches list
# last 20 matches list -> remove duplicates [m = total length of match list]
# [n+m queries] for each match look up match information
# break down match information




In [ ]:
# Test which is faster for the API 
# summoner_request or summonerId

from functools import wraps
from time import time


def timing(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print 'Elapsed time: {}'.format(end-start)
        return result
    return wrapper





In [4]:
# How to make a request for username
# use headers variable always? 
# NA only, probably different <>.api.riotgames.com 

summoner_request = "AutoBattleBot"
search_summoner = 'https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/'+summoner_request


response_search_summoner = requests.get(search_summoner, headers=headers)
print(API_Responses[response_search_summoner.status_code])

# Should have some sort of verifcation
# Should have some sort of error code to response

OK


In [5]:
print(response_search_summoner.json())

{'id': 'eLVSC31XyJpUcssw1Xx14DeMCaXZNdXqM3N2qgNTTD1tzDD8', 'accountId': '9saQlcagNsGYGvRvGr8gZrvm1JdJRlQXQpQ2qVgdqYzxV1lee1ce_13l', 'puuid': 'PvVcIk6ij_ib-lFOp-euC__GoPz4GTim3dMXe1XdzBAEDXBgL5ElnUSJRBDNSE2_qAb2qULSoEDVjA', 'name': 'AutoBattleBot', 'profileIconId': 29, 'revisionDate': 1603349358000, 'summonerLevel': 1}


In [6]:
r = requests.head(url=search_summoner)
print(r.headers)

{'Content-Type': 'application/json;charset=utf-8', 'Date': 'Fri, 11 Dec 2020 05:30:22 GMT, Fri, 11 Dec 2020  05:30:22 GMT', 'X-Riot-Edge-Trace-Id': '7da30b49-7b4a-4e1f-9688-bedd515c5a25', 'Connection': 'keep-alive'}


In [27]:
# How we convert to JSON and parse through
# Guessing it should be fine because each header is easily searched on RIOT API and they won't change

response_search_summoner_dict = response_search_summoner.json()
# Player Universally Unique IDentifiers (newer way)
puuid = response_search_summoner_dict['puuid']
# summoner id - mroe region based identification (old way)
summoner_id = response_search_summoner_dict['id']

print("puuid: " + puuid)
print("summoner_id: " + summoner_id)

puuid: PvVcIk6ij_ib-lFOp-euC__GoPz4GTim3dMXe1XdzBAEDXBgL5ElnUSJRBDNSE2_qAb2qULSoEDVjA
summoner_id: eLVSC31XyJpUcssw1Xx14DeMCaXZNdXqM3N2qgNTTD1tzDD8


In [28]:
# Request ranking information based on summoner id

search_ranking = "https://na1.api.riotgames.com/tft/league/v1/entries/by-summoner/" + summoner_id

response_search_ranking = requests.get(search_ranking, headers=headers)
print(API_Responses[response_search_ranking.status_code])


OK


In [15]:
# returns a list! instead of a dictionary... so we'll need to fix it....
print(response_search_ranking.json()[0])

{'leagueId': '41343c48-50bd-4175-a877-95770e208f04', 'queueType': 'RANKED_TFT', 'tier': 'PLATINUM', 'rank': 'I', 'summonerId': 'MnEQ6BcFXhipHCnnuM344LZ8bJ7rqkN61pXGhmPWMw-k5PqW', 'summonerName': '88chwain', 'leaguePoints': 0, 'wins': 26, 'losses': 165, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}


In [16]:
response_search_ranking_dict = response_search_ranking.json()[0]
tier = response_search_ranking_dict['tier']
rank = response_search_ranking_dict['rank']
LP = response_search_ranking_dict['leaguePoints']
wins = response_search_ranking_dict['wins']
losses = response_search_ranking_dict['losses']

#unsure about these metrics
#veteran = response_search_ranking_dict['veteran']
#inactive = response_search_ranking_dict['inactive']
#freshBlood = response_search_ranking_dict['freshBlood']
#hotStreak = response_search_ranking_dict['hotStreak']

In [17]:
# Request for Match IDs based on puuid
n_recent_matches = 20
search_match = "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + puuid + "/ids?count="+ str(n_recent_matches)

response_search_match = requests.get(search_match, headers=headers)
print(API_Responses[response_search_match.status_code])

OK


In [18]:
response_search_match_dict = response_search_match.json()
# Player Unique User Identification -Used instead of raw name
response_search_match_dict

match_list = dict()
# From 0 to n_recent_matches
for number, match_id in enumerate(response_search_match_dict):
    match_list[number]= match_id


In [19]:
match_list
#match_list[0]

{0: 'NA1_3630152875',
 1: 'NA1_3628829278',
 2: 'NA1_3628851759',
 3: 'NA1_3627838012',
 4: 'NA1_3627812452',
 5: 'NA1_3627501044',
 6: 'NA1_3626440134',
 7: 'NA1_3626341535',
 8: 'NA1_3626174069',
 9: 'NA1_3625892017',
 10: 'NA1_3624488945',
 11: 'NA1_3624530116',
 12: 'NA1_3624442500',
 13: 'NA1_3623568534',
 14: 'NA1_3623510920',
 15: 'NA1_3623422925',
 16: 'NA1_3623345978',
 17: 'NA1_3623289356',
 18: 'NA1_3623081923',
 19: 'NA1_3622906182'}

In [20]:
# Match Information
search_match_info = "https://americas.api.riotgames.com/tft/match/v1/matches/" + match_list[0]

response_search_match_info = requests.get(search_match_info, headers=headers)
print(API_Responses[response_search_match_info.status_code])

OK


In [21]:
response_search_match_info_dict = response_search_match_info.json()

In [22]:
print(response_search_match_info_dict)

{'metadata': {'data_version': '5', 'match_id': 'NA1_3630152875', 'participants': ['pLp88KkYvcbTnKevXUb33CMol_eJSWq4xHjgINLW5iegO3Wtg0sUSy6LSJe44HQasMW90D_90-4C3w', 'OmFkLrQBN-xs0haPk5ooh-g2Of5Or7nqyQlEh0G7ed2fTVYErqbfylYYKu7PHSMGWQ0jSluxKC3hFw', 'JzKl_pACMSDQ_bY38GvOAdCu62yAnU9AUyiTnwJThphb5X4HmLC0GiV7LP9zxXItY139Eri1blg3aw', 'ARzZ8ZIsfhefpafFh9gvDLXbpA6Ws30QDbmhoUs8K6xsw1GGVmhEHwWCuUw7TQ5a5cjU15p9VX7XWQ', '_ILPMfKQrbeIN6auqksi_ajKmXaiLqeSQyvZlAOycAGqlLzN8XlpmiMvIUfJ_8lx1Ag8ZE7lEOKIDA', 'ZgaJ0EurUEW_InpChbfGMZt6Oda24huCDe1deMX70ltmvMjQaARxoYnPv8JDi9xQGLRCOj20Rfn2Rw', 'BsTuu6EscV1ofuNOBzBIQMaYOaglXrAumTmjGWKq8CdcU3cmSxZczwcntma8eimh14JpYnVDm-6Rkg', 'idjfj6nouxMYDnkAbsUrzJyjYVcfMu7BkLKAtVXIZGsdTdU-22sevGQEqLJmYa8AgE3jcfi0oACcmQ']}, 'info': {'game_datetime': 1603498465558, 'game_length': 2052.78662109375, 'game_version': 'Version 10.21.339.2173 (Oct 09 2020/15:54:03) [PUBLIC] <Releases/10.21>', 'participants': [{'companion': {'content_ID': '64691b63-1603-443e-bde5-727148f440ed', 'skin_ID'

In [23]:
import pprint
pp = pprint.PrettyPrinter(depth=8)
pp.pprint(response_search_match_info_dict)

{'info': {'game_datetime': 1603498465558,
          'game_length': 2052.78662109375,
          'game_version': 'Version 10.21.339.2173 (Oct 09 2020/15:54:03) '
                          '[PUBLIC] <Releases/10.21>',
          'participants': [{'companion': {'content_ID': '64691b63-1603-443e-bde5-727148f440ed',
                                          'skin_ID': 4,
                                          'species': 'PetMiner'},
                            'gold_left': 1,
                            'last_round': 35,
                            'level': 8,
                            'placement': 2,
                            'players_eliminated': 0,
                            'puuid': 'pLp88KkYvcbTnKevXUb33CMol_eJSWq4xHjgINLW5iegO3Wtg0sUSy6LSJe44HQasMW90D_90-4C3w',
                            'time_eliminated': 2044.4974365234375,
                            'total_damage_to_players': 122,
                            'traits': [{'name': 'Cultist',
                                   

In [25]:
pp.pprint(response_search_match_info_dict['info']['participants'][0])

{'companion': {'content_ID': '64691b63-1603-443e-bde5-727148f440ed',
               'skin_ID': 4,
               'species': 'PetMiner'},
 'gold_left': 1,
 'last_round': 35,
 'level': 8,
 'placement': 2,
 'players_eliminated': 0,
 'puuid': 'pLp88KkYvcbTnKevXUb33CMol_eJSWq4xHjgINLW5iegO3Wtg0sUSy6LSJe44HQasMW90D_90-4C3w',
 'time_eliminated': 2044.4974365234375,
 'total_damage_to_players': 122,
 'traits': [{'name': 'Cultist',
             'num_units': 2,
             'style': 0,
             'tier_current': 0,
             'tier_total': 3},
            {'name': 'Fortune',
             'num_units': 1,
             'style': 0,
             'tier_current': 0,
             'tier_total': 2},
            {'name': 'Keeper',
             'num_units': 1,
             'style': 0,
             'tier_current': 0,
             'tier_total': 3},
            {'name': 'Set4_Adept',
             'num_units': 1,
             'style': 0,
             'tier_current': 0,
             'tier_total': 3},
        

In [28]:
response_search_match_info_dict['info']['participants'][0]['gold_left']
response_search_match_info_dict['info']['participants'][0]['last_round']
response_search_match_info_dict['info']['participants'][0]['level']
response_search_match_info_dict['info']['participants'][0]['placement']
response_search_match_info_dict['info']['participants'][0]['players_eliminated']
response_search_match_info_dict['info']['participants'][0]['puuid']
response_search_match_info_dict['info']['participants'][0]['time_eliminated']
response_search_match_info_dict['info']['participants'][0]['total_damage_to_players']


122

In [29]:
trait_tier = dict()
for trait in response_search_match_info_dict['info']['participants'][0]['traits']:
    trait_tier[trait['name']] =  trait['tier_current']

In [30]:
trait_tier

{'Cultist': 0,
 'Fortune': 0,
 'Keeper': 0,
 'Set4_Adept': 0,
 'Set4_Assassin': 2,
 'Set4_Mystic': 0,
 'Set4_Ninja': 2,
 'Set4_Shade': 2,
 'Set4_Tormented': 1,
 'Warlord': 0}

In [32]:
trait_tier_sorted = sorted(trait_tier.items(),  key=lambda x:x[1], reverse=True)
trait_tier_sorted

[('Set4_Assassin', 2),
 ('Set4_Ninja', 2),
 ('Set4_Shade', 2),
 ('Set4_Tormented', 1),
 ('Cultist', 0),
 ('Fortune', 0),
 ('Keeper', 0),
 ('Set4_Adept', 0),
 ('Set4_Mystic', 0),
 ('Warlord', 0)]

In [33]:
trait_tier_clean = dict(filter(lambda num: num[1] !=0, trait_tier_sorted))
trait_tier_clean

{'Set4_Assassin': 2, 'Set4_Ninja': 2, 'Set4_Shade': 2, 'Set4_Tormented': 1}

In [73]:
type(response_search_match_info_dict['info']['participants'][0]['units'])

list

In [74]:
type(response_search_match_info_dict['info']['participants'][0]['units'][0])

dict

In [78]:
response_search_match_info_dict['info']['participants'][0]['units'][0]['chosen']

'Sharpshooter'

In [34]:
newlist = sorted(response_search_match_info_dict['info']['participants'][0]['units'], key=lambda k: len(k['items']), reverse=True) 

In [35]:
newlist

[{'character_id': 'TFT4_Akali',
  'chosen': 'Set4_Assassin',
  'items': [22, 19, 44],
  'name': '',
  'rarity': 2,
  'tier': 3},
 {'character_id': 'TFT4_Katarina',
  'items': [45, 69, 13],
  'name': '',
  'rarity': 2,
  'tier': 2},
 {'character_id': 'TFT4_Kennen',
  'items': [35, 78],
  'name': '',
  'rarity': 2,
  'tier': 2},
 {'character_id': 'TFT4_Evelynn',
  'items': [47],
  'name': '',
  'rarity': 2,
  'tier': 2},
 {'character_id': 'TFT4_Shen',
  'items': [25],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT4_Zed', 'items': [], 'name': '', 'rarity': 1, 'tier': 2},
 {'character_id': 'TFT4_Pyke',
  'items': [],
  'name': '',
  'rarity': 1,
  'tier': 1},
 {'character_id': 'TFT4_Kayn',
  'items': [],
  'name': '',
  'rarity': 4,
  'tier': 1}]

In [36]:
itemization_dict = {
    0: "Not Itemized",
    1: "Partially Itemized",
    2: "Semi Itemized",
    3: "Fully Itemized"
    
}

In [37]:
chosen_info = ""

for i in newlist:
    if 'chosen' in i:
#        print(1)
        #print(i['character_id'])
        #print(i['tier'])
        chosen_info += i['chosen'] + " "
        chosen_info +=i ['character_id'] + " "
        if i['tier'] > 2:
            chosen_info += "Upgraded" + " "
        #print(i['items'])
        chosen_info += itemization_dict[len(i['items'])]
        
        print(chosen_info)
            

Set4_Assassin TFT4_Akali Upgraded Fully Itemized


In [55]:
response_search_match_info_dict['info']['participants'][0]['units'] # this is a list

response_search_match_info_dict['info']['participants'][0]['units'][0] #this is a dict


# List of units order by 
#chosen (Save this as variable for naming purposes later) # Regex
#number of items (count this list)
#tier
#rarity



[{'character_id': 'TFT4_Nidalee',
  'chosen': 'Sharpshooter',
  'items': [99, 35, 23],
  'name': '',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT4_Vayne',
  'items': [],
  'name': '',
  'rarity': 0,
  'tier': 1},
 {'character_id': 'TFT4_Teemo',
  'items': [37],
  'name': '',
  'rarity': 1,
  'tier': 3},
 {'character_id': 'TFT4_Thresh',
  'items': [],
  'name': '',
  'rarity': 1,
  'tier': 3},
 {'character_id': 'TFT4_Jinx',
  'items': [23, 14, 69],
  'name': '',
  'rarity': 2,
  'tier': 2},
 {'character_id': 'TFT4_Sejuani',
  'items': [37, 39, 44],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT4_Jhin',
  'items': [17, 35, 12],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT4_Azir',
  'items': [22],
  'name': '',
  'rarity': 4,
  'tier': 1}]

SQL TABLES

Username Information

Ranking Information over time
? how does this line up with time of match
* Probably just for production-ready apps, not available for now


Match History


Match information
Columns?

* Match ID
* puuid
* placing
* level
* Comp
* Chosen - style
* Units and Items
* gold_left
* last_round

* players_eliminated
* time_eliminated
* total_damage_to_players



In [ ]:
# codify last round into stage-round format
# units ordered by tier and number of items
    # codify items based on id
    # how to determine chosen? chosen tag in 'units'!
# codify traits into compositions? 
    # this person is going sharpshooters because.... traits + items on certain units


In [1]:
import sqlalchemy as db

# how to connect to local? I think I need a driver to do so
# https://www.youtube.com/watch?v=qfYzCdkVwig
# pip installed pymysql
# https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91

In [2]:
db.__version__

'1.3.16'

In [3]:
import pymysql

In [4]:
engine = db.create_engine("mysql+pymysql://root@localhost/TFT", echo = True)

In [5]:
metadata = db.MetaData()
conn = engine.connect()

2020-12-01 18:59:01,226 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2020-12-01 18:59:01,228 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:01,231 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2020-12-01 18:59:01,232 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:01,233 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2020-12-01 18:59:01,234 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:01,235 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2020-12-01 18:59:01,235 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:01,237 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2020-12-01 18:59:01,237 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:01,238 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2020-12-01 18:59:01,239 INFO sqlalchemy.engine.base.E

In [6]:
meta = db.MetaData()
# engine.execute("CREATE TABLE items (item_id int, item_name varchar(255))")
items = db.Table(
    'items', meta, 
    db.Column('item_id', db.Integer, primary_key = True, autoincrement = False), 
    db.Column('item_name', db.String(255))
)
#engine.execute("CREATE TABLE match_id (match_id int, game_datetime datetime, game_length int, game_version varchar(255))")
match_id = db.Table(
    'match_id', meta,
    db.Column('match_id', db.Integer, primary_key = True, autoincrement = False),
    db.Column('game_datetime', db.DateTime),
    db.Column('game_length', db.Integer, autoincrement = False),
    db.Column('game_version', db.String(255))
)

#engine.execute("CREATE TABLE match_info (match_id int, puuid varchar(255), placement int, level int, "
#                                       "time_eliminated int, last_round int, gold_left int, total_damage_to_players int,"
#                                        "traits varchar(255), chosen_info varchar(255), units varchar(255))")


match_info = db.Table(
    'match_info', meta,
    db.Column('match_id', db.Integer, primary_key = True, autoincrement = False),
    db.Column('puuid', db.String(255)),
    db.Column('placement', db.Integer, autoincrement = False),
    db.Column('level', db.Integer, autoincrement = False),
    db.Column('time_eliminated', db.Integer, autoincrement = False),
    db.Column('gold_left', db.Integer, autoincrement = False),
    db.Column('total_damage_to_players', db.Integer, autoincrement = False),
    db.Column('traits', db.String(255)),
    db.Column('chosen_info', db.String(255)),
    db.Column('units', db.String(255))
)

#engine.execute("CREATE TABLE user_info (puuid varchar(255), summoner_id varchar(255), summoner_username varchar(255))")
user_info = db.Table(
    'user_info', meta,
    db.Column('puuid', db.String(255), index=True),
    db.Column('summoner_id', db.String(255), index=True),
    db.Column('summoner_username', db.String(255)),
    db.UniqueConstraint('puuid')
)

#engine.execute("CREATE TABLE user_rank (summoner_id varchar(255), tier varchar(255), rank varchar(255), league_points int, revision_date datetime)")
user_rank = db.Table(
    'user_rank', meta,
    db.Column('summoner_id', db.String(255)),
    db.Column('tier', db.String(255)),
    db.Column('rank', db.String(255)),
    #needs some time value that represents when the match occured
)


meta.create_all(engine)

2020-12-01 18:59:04,745 INFO sqlalchemy.engine.base.Engine DESCRIBE `items`
2020-12-01 18:59:04,746 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:04,771 INFO sqlalchemy.engine.base.Engine DESCRIBE `match_id`
2020-12-01 18:59:04,772 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:04,788 INFO sqlalchemy.engine.base.Engine DESCRIBE `match_info`
2020-12-01 18:59:04,789 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:04,804 INFO sqlalchemy.engine.base.Engine DESCRIBE `user_info`
2020-12-01 18:59:04,805 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 18:59:04,822 INFO sqlalchemy.engine.base.Engine DESCRIBE `user_rank`
2020-12-01 18:59:04,823 INFO sqlalchemy.engine.base.Engine {}


In [9]:

query = db.select([user_info])
ResultProxy = conn.execute(query)

ResultSet = ResultProxy.fetchall()
ResultSet

2020-12-01 19:06:14,328 INFO sqlalchemy.engine.base.Engine SELECT user_info.puuid, user_info.summoner_id, user_info.summoner_username 
FROM user_info
2020-12-01 19:06:14,328 INFO sqlalchemy.engine.base.Engine {}


[('1', 'A', 'AA'),
 ('2', 'B', 'BB'),
 ('3', 'C', 'CC'),
 ('4', 'D', 'DD'),
 ('7', 'A', 'AA')]

In [48]:
# https://docs.sqlalchemy.org/en/14/core/dml.html
# Inserting
# if one of them is already found, then all of it wont work
try:
    conn.execute(user_info.insert(), [
        {'puuid': 7, 'summoner_id': 'A', 'summoner_username':'AA'}
     ])
except db.exc.IntegrityError:
    print("An IntegrityError exception occurred. Already in Database") 

2020-12-01 21:15:55,601 INFO sqlalchemy.engine.base.Engine INSERT INTO user_info (puuid, summoner_id, summoner_username) VALUES (%(puuid)s, %(summoner_id)s, %(summoner_username)s)
2020-12-01 21:15:55,602 INFO sqlalchemy.engine.base.Engine {'puuid': 7, 'summoner_id': 'A', 'summoner_username': 'AA'}
2020-12-01 21:15:55,608 INFO sqlalchemy.engine.base.Engine ROLLBACK
An IntegrityError exception occurred. Already in Database


In [30]:
# sel = select([literal("1"), literal("John")]).where(
#            ~exists([example_table.c.id]).where(example_table.c.id == 1)
#       )
sel = db.select([db.literal('7'), db.literal('A'), db.literal('AA')]).where(
           ~db.exists([user_info.c.puuid]).where(user_info.c.puuid == '7')
      )

ins = user_info.insert().from_select(['puuid', 'summoner_id','summoner_username'], sel)
#print(ins)
conn.execute(ins)
#possibly would need to have single inserts that could break, or someway to handle itegrity errors per each of the multiple
#or some session merge thing




2020-12-01 19:46:10,006 INFO sqlalchemy.engine.base.Engine INSERT INTO user_info (puuid, summoner_id, summoner_username) SELECT %(param_1)s AS anon_1, %(param_2)s AS anon_2, %(param_3)s AS anon_3 
WHERE NOT (EXISTS (SELECT user_info.puuid 
FROM user_info 
WHERE user_info.puuid = %(puuid_1)s))
2020-12-01 19:46:10,007 INFO sqlalchemy.engine.base.Engine {'param_1': '7', 'param_2': 'A', 'param_3': 'AA', 'puuid_1': '7'}
2020-12-01 19:46:10,008 INFO sqlalchemy.engine.base.Engine ROLLBACK


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'WHERE NOT (EXISTS (SELECT user_info.puuid \nFROM user_info \nWHERE user_info.puuid' at line 2")
[SQL: INSERT INTO user_info (puuid, summoner_id, summoner_username) SELECT %(param_1)s AS anon_1, %(param_2)s AS anon_2, %(param_3)s AS anon_3 
WHERE NOT (EXISTS (SELECT user_info.puuid 
FROM user_info 
WHERE user_info.puuid = %(puuid_1)s))]
[parameters: {'param_1': '7', 'param_2': 'A', 'param_3': 'AA', 'puuid_1': '7'}]
(Background on this error at: http://sqlalche.me/e/f405)

In [41]:
conn.execute(user_info.delete())

2020-11-30 20:39:11,814 INFO sqlalchemy.engine.base.Engine DELETE FROM user_info
2020-11-30 20:39:11,815 INFO sqlalchemy.engine.base.Engine {}
2020-11-30 20:39:11,816 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
user_info.drop(engine)

2020-11-30 20:43:18,498 INFO sqlalchemy.engine.base.Engine 
DROP TABLE user_info
2020-11-30 20:43:18,498 INFO sqlalchemy.engine.base.Engine {}


In [33]:
m = db.MetaData()

example_table = db.Table("example_table", m,
                        db.Column('id', db.Integer),
                        db.Column('name', db.String)
                    )

sel = db.select([db.literal("1"), db.literal("John")]).where(
           ~db.exists([example_table.c.id]).where(example_table.c.id == 1)
      )

ins = example_table.insert().from_select(["id", "name"], sel)
print(ins)
conn.execute(ins)

INSERT INTO example_table (id, name) SELECT :param_1 AS anon_1, :param_2 AS anon_2 
WHERE NOT (EXISTS (SELECT example_table.id 
FROM example_table 
WHERE example_table.id = :id_1))
2020-12-01 19:49:49,518 INFO sqlalchemy.engine.base.Engine INSERT INTO example_table (id, name) SELECT %(param_1)s AS anon_1, %(param_2)s AS anon_2 
WHERE NOT (EXISTS (SELECT example_table.id 
FROM example_table 
WHERE example_table.id = %(id_1)s))
2020-12-01 19:49:49,519 INFO sqlalchemy.engine.base.Engine {'param_1': '1', 'param_2': 'John', 'id_1': 1}
2020-12-01 19:49:49,521 INFO sqlalchemy.engine.base.Engine ROLLBACK


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'WHERE NOT (EXISTS (SELECT example_table.id \nFROM example_table \nWHERE example_ta' at line 2")
[SQL: INSERT INTO example_table (id, name) SELECT %(param_1)s AS anon_1, %(param_2)s AS anon_2 
WHERE NOT (EXISTS (SELECT example_table.id 
FROM example_table 
WHERE example_table.id = %(id_1)s))]
[parameters: {'param_1': '1', 'param_2': 'John', 'id_1': 1}]
(Background on this error at: http://sqlalche.me/e/f405)

In [34]:
sel = select(table1.c.a, table1.c.b).where(table1.c.c > 5)
ins = table2.insert().from_select(['a', 'b'], sel)

NameError: name 'select' is not defined

In [37]:
# How to drop tables completely
# Question: Transactional? In SQLAlchemy this is known as "sessions"
#students.drop(engine)


NameError: name 'items' is not defined

In [39]:
# Create table 
# Insert Values
# Repeatedly insert values

https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
    https://leportella.com/sqlalchemy-tutorial/
        https://www.michaelcho.me/article/sqlalchemy-commit-flush-expire-refresh-merge-whats-the-difference
            Sessions
            #https://docs.sqlalchemy.org/en/14/orm/session_basics.html#session-faq-whentocreate

SyntaxError: invalid syntax (<ipython-input-39-cdeddc0e9e93>, line 5)